In [1]:
import datetime
from dateutil.relativedelta import relativedelta
import collections
import pandas as pd

import dwsemear
import postgresemear as sql # mover os arquivos para a pasta principal

In [12]:
def adicione_liquidacoes(contratos_list, dados_liquidacao):
    lista_liquidacoes = []
    for contrato in contratos_list:
        dados_contrato = dados_liquidacao[dados_liquidacao.Numero_do_contrato == contrato].copy()
        for i in dados_contrato.index:
            lista_liquidacoes.append(dados_contrato.loc[i,:].tolist())
    return lista_liquidacoes

# ESSA FUNCAO NAO ESTA SENDO UTILIZADA
def pega_inadimplencia_3065(dados_pdd):
    dados_3065 = dados_pdd[dados_pdd.CodigoDoProduto.isin([3065])].copy()
    total = len(dados_3065)
    inadimplentes = len(dados_3065[dados_3065.DiasEmAtraso > 60])
    print('inadimplencia:  ', inadimplentes/total)

def diferenca_de_datas_em_meses(data_inicial, data_final):
    r = relativedelta(data_final, data_inicial) 
    return r.months + 12 * r.years

def parcelas_abertas(data_inicial, data_final):
    if data_final < data_inicial:
        return 0
    return diferenca_de_datas_em_meses(data_inicial, data_final) + 1
 
def numero_parcelas_pagas(liquidacoes, contrato, data_entrada):
    linhas_contrato = liquidacoes[liquidacoes.Numero_do_contrato == contrato].copy()
    linhas_contrato.Data_de_liquidacao = linhas_contrato.Data_de_liquidacao.apply(remove_localize)
    if len(linhas_contrato) == 0:
        return 0
    parcelas_pagas = len(linhas_contrato[linhas_contrato.Data_de_liquidacao <= data_entrada])
    return parcelas_pagas    
    
def valor_pago(liquidacoes, contrato, data_entrada):
    linhas_contrato = liquidacoes[liquidacoes.Numero_do_contrato == contrato].copy()
    linhas_contrato.Data_de_liquidacao = linhas_contrato.Data_de_liquidacao.apply(remove_localize)
    if len(linhas_contrato) == 0:
        return 0
    linhas_contrato = linhas_contrato[linhas_contrato.Data_de_liquidacao <= data_entrada]
    if len(linhas_contrato) == 0:
        return 0
    if linhas_contrato.Codigo_renegociacao_contrato.mean(): # JOGANDO FORA QUEM FOI RENEGOCIADO
        print('Descartando renegociado:  ', contrato)
        return 0
    return sum(linhas_contrato.Valor_recebido_na_liquidacao)

def enriquecimento_plano_parcela(linha_pdd):
    data_inicial = linha_pdd['DataDaProximaParcela'].iloc[0]
    data_final = linha_pdd['DataDeVencimento'].iloc[0]
    plano = diferenca_de_datas_em_meses(data_inicial, data_final) + 2
    data_primeira_parcela = data_inicial + relativedelta(months=-1)
    valor_parcela = linha_pdd.ValorDaProximaParcela.iloc[0]
    return [plano, data_primeira_parcela, valor_parcela]
    
# ATENCAO - precisa rever essa funcao depois.
def remove_localize_x(data):  # esse localize esta relacionado a fuso horario
    return data.tz_localize(None)
    
def remove_localize(data):    
    return data.replace(tzinfo=None)
    
#SOMAR 1 MES    
#datetime.datetime(2019,2,27,23,59) + relativedelta(months=+1)#

# OBTENCAO DOS NUMEROS DE CONTRATOS QUE ORIGINARAM EM UMA DADA DATA

In [3]:
CODIGOS_EP = [3086]
#CODIGOS_EP = [3064, 3065, 3069, 3070, 3071, 3072, 3077, 
#              3078, 3081, 3086, 3087, 3088, 3092, 3095, 
#              3096, 3103, 3105]
COLUNAS_PDD = ['ID',
            'DataDeReferencia',
            'DataDeOriginiacao',
            'DataDaProximaParcela',
            'DataDeVencimento',
            'DataDeRenegociacao',
            'CPF',
            'CodigoDoProduto',
            'CodigoDoContrato',
            'DiasEmAtraso',
            'QuantidadeDeParcelasAbertas',
            'ValorSaldo',
            'ValorPDD',
            'ValorOperacao',
            'RendaDoCliente',
            'ValorDaProximaParcela',
            'RatingContabilFim']# os dados dessas querys nao vem ordenados
parametros = (datetime.datetime(2019,1,31),
              datetime.datetime(2018,12,1), 
              datetime.datetime(2018,12,31,23,59))
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      AND DataDeOriginiacao BETWEEN %s AND %s
                      ''',
                      parametros)
dados_pdd.columns = COLUNAS_PDD
dados_EP = dados_pdd[dados_pdd.CodigoDoProduto.isin(CODIGOS_EP)].copy()

In [3]:
COLUNAS_PDD = ['ID',
            'DataDeReferencia',
            'DataDeOriginiacao',
            'DataDaProximaParcela',
            'DataDeVencimento',
            'DataDeRenegociacao',
            'CPF',
            'CodigoDoProduto',
            'CodigoDoContrato',
            'DiasEmAtraso',
            'QuantidadeDeParcelasAbertas',
            'ValorSaldo',
            'ValorPDD',
            'ValorOperacao',
            'RendaDoCliente',
            'ValorDaProximaParcela',
            'RatingContabilFim']# os dados dessas querys nao vem ordenados
parametros = (datetime.datetime(2019,1,31),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',
                      parametros)
dados_pdd.columns = COLUNAS_PDD

In [4]:
jan = dados_pdd

In [13]:
jan.DataDeOriginiacao = jan.loc[:,'DataDeOriginiacao'].apply(remove_localize)

In [21]:
jan_on = jan[(jan.DataDeOriginiacao >= datetime.datetime(2019,1,1)) &
    (jan.DataDeOriginiacao <= datetime.datetime(2019,1,31))].copy()

In [18]:
jan[jan.CodigoDoContrato == 54796133]

,ID,DataDeReferencia,DataDeOriginiacao,DataDaProximaParcela,DataDeVencimento,DataDeRenegociacao,CPF,CodigoDoProduto,CodigoDoContrato,DiasEmAtraso,QuantidadeDeParcelasAbertas,ValorSaldo,ValorPDD,ValorOperacao,RendaDoCliente,ValorDaProximaParcela,RatingContabilFim
208333,208333,2019-01-31 00:00:00-03:00,2019-01-02,2019-02-01 00:00:00-03:00,2020-02-01 00:00:00-03:00,None,23526521204,3090,54796133,0,13,830.03,4.15,768.5,1683.0,99.9,A


In [24]:
len(jan_on)

32754

In [27]:
19614 + 1599 + 17 + 13125

34355

# Corte de informacoes importantes do PDD

In [4]:
contratos_dez = list(set(dados_EP.CodigoDoContrato))

In [5]:
len(contratos_dez) # Book deu 322

2280

In [6]:
enriquecimento = {}
for contrato in contratos_dez:
    linha_pdd = dados_EP[dados_EP.CodigoDoContrato == contrato]
    plano, data_primeira_parcela, valor_parcela = enriquecimento_plano_parcela(linha_pdd)
    enriquecimento[contrato] = {}
    enriquecimento[contrato]['plano'] = plano
    enriquecimento[contrato]['primeira'] = data_primeira_parcela.tz_localize(None)
    enriquecimento[contrato]['valor'] = valor_parcela    

# ANDE NOS CONTRATOS, VEJA QUANTO DEVIA E QUANTO PAGOU
- De carga nas liquidacoes abaixo e volte aqui

In [9]:
liq = corte_nas_liquidacoes(contratos_dez)

In [24]:
k = 0
data_entrada = datetime.datetime(2019,6,14)  
inadimplentes = 0
total_receber = 0
total_pago = 0
for contrato in enriquecimento:
    primeira = enriquecimento[contrato]['primeira']
    plano = enriquecimento[contrato]['plano']
    receber = parcelas_abertas(primeira, data_entrada)
    pagas = numero_parcelas_pagas(liq, contrato, data_entrada)
    if receber > plano:
        receber = plano
        #if pagas == receber:
        #    continue
        
    valor = enriquecimento[contrato]['valor']
    total_receber += receber * valor
    #print(primeira, 'pgas: ', pagas, 'reber: ', receber, valor)
    #if pagas > receber:
    #    pagas = receber
    total_pago += valor_pago(liq, contrato, data_entrada)
    if pagas <= receber - 2:
        inadimplentes += 1
    #k+=1
    #if k > 15: break
    

Descartando renegociado:   54782051
Descartando renegociado:   54724748
Descartando renegociado:   54759005
Descartando renegociado:   54770501


In [29]:
total_receber # se vc sumir com contratos liquidados este numero nao bate

3270563.1599999876

In [30]:
total_pago

2877026.200000002

In [31]:
total_receber - total_pago

393536.9599999855

In [32]:
(total_receber - total_pago)/total_receber

0.1203269714564959

In [ ]:
inadimplentes/2280

In [228]:
150950.31/659588.46

0.22885529258653192

In [242]:
# Tenho 2280 contratos - book -> 2337
# eu tenho:
print(int(100*(2337-2280)/2337),'%')
print(2337 - 2280)

2 %
57


In [243]:
total_aberto = total_receber - total_pago
total_aberto_ref = 526511.14
total_venc = total_receber
total_venc_ref = 3353287.28

In [246]:
# meu vencimento foi 82 mil menor
# Tive 57 menos contratos, e mais ou menos aceitavel.
total_venc - total_venc_ref 

-82724.12000001222

In [247]:
# meu calculo de vencimentos esta bem maior do que deveria.
total_aberto - total_aberto_ref

-132974.1800000145

# CARREGANDO LIQUIDACOES

In [8]:
def corte_nas_liquidacoes(contratos_dez):
    COLUNAS_LIQUIDACAO = [
            'ID',
            'Data_de_liquidacao',
            'Data_de_vencimento_da_parcela_liquidada',
            'CPF',
            'Numero_do_contrato',
            'Codigo_renegociacao_contrato',
            'Codigo_do_produto',
            'Parcela',
            'Dias_em_atraso_na_hora_da_liquidacao',
            'Valor_recebido_na_liquidacao',
            'Codigo_forma',
            'Situacao_da_liquidacao',
            'Descricao_forma',
            'Estornado']

    lista_liquidacoes = []    
    parametros = (datetime.datetime(2018,1,1), datetime.datetime(2019,1,31,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    parametros = (datetime.datetime(2019,2,1), datetime.datetime(2019,2,28,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    parametros = (datetime.datetime(2019,3,1), datetime.datetime(2019,3,31,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    parametros = (datetime.datetime(2019,4,1), datetime.datetime(2019,4,30,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    parametros = (datetime.datetime(2019,5,1), datetime.datetime(2019,5,31,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    parametros = (datetime.datetime(2019,6,1), datetime.datetime(2019,6,30,23,59))
    dados_liquidacao = sql.query_para_pandas('''SELECT *
                          FROM liquidacoes
                          WHERE Data_de_liquidacao BETWEEN %s AND %s
                          ''',parametros)
    dados_liquidacao.columns = COLUNAS_LIQUIDACAO
    lista_liquidacoes += adicione_liquidacoes(contratos_dez, dados_liquidacao)    
    liq = pd.DataFrame(lista_liquidacoes)
    liq.columns = COLUNAS_LIQUIDACAO
    return liq
       